In [1]:
import datetime
import os
import random
import time
import gc
import sys
import joblib
import numpy as np
import torch
from models import *
from base import *
from torch.utils.data import DataLoader
from utils import check_mkdir, evaluate,CrossEntropyLoss2d
import warnings
warnings.filterwarnings("ignore")

In [2]:
def fit_gmm_model(feat_np, true_np, prds_np, cl, n_components):
    
    model = mixture.GaussianMixture(n_components=n_components, verbose=2, covariance_type='tied', init_params='random', reg_covar=reg_covar)
    
    cl_feat_flat = feat_np[(true_np == cl) & (prds_np == cl), :]
    
    model.fit(cl_feat_flat)
    return model

def pred_pixelwise(model_list, feat_np, prds_np, num_known_classes, threshold):
    
    scores = np.zeros_like(prds_np, dtype=np.float)
    for c in range(num_known_classes):
        feat_msk = (prds_np == c)
        if np.any(feat_msk):
            scores[feat_msk] = model_list[c].score_samples(feat_np[feat_msk, :])
        
    prds_np[scores < threshold] = num_known_classes
    return prds_np, scores

def load_models(num_known_classes, outp_path, exp_group, exp_name):
    model_list = []
    for c in range(num_known_classes):
        load_model = os.path.join(outp_path, exp_group, exp_name, 'model_' + str(c) + '.pkl')
        model_list.append(joblib.load(load_model))
    return model_list

In [3]:
## Predefining directories.
ckpt_path = './ckpt'
outp_path = './outputs'
imag_path = './images'

# Setting predefined arguments.
args = {
    'epoch_num': 1200,            # Number of epochs.
    'lr': 1e-4,                   # Learning rate.
    'weight_decay': 5e-4,         # L2 penalty.
    'momentum': 0.9,              # Momentum.
    'num_workers': 2,             # Number of workers on data loader.
    'print_freq': 1,              # Printing frequency for mini-batch loss.
    'w_size': 224,                # Width size for image resizing.
    'h_size': 224,                # Height size for image resizing.
    'test_freq': 1200,            # Test each test_freq epochs.
    'save_freq': 1200,            # Save model each save_freq epochs.
    'open_threshold': -50,        # Threshold for OpenSet.
    'n_components': 16,           # Numnber of components for dimensionality reduction. (4, 8, 16)
    'num_classes': 5,             # Number of classes.
}


- conv_name -> name of the backbone (unet, fcnwideresnet50, fcndensenet121)
- args['hidden_classes'] -> UUCs (0,1,2,3,4)
- epoch -> vaihingen was treined for 1200 epochs and Potsdam for 600
- args['n_components'] -> 4, 8 or 16 yields similar results
- reg_covar = 0.0001 for Vaihingen

In [4]:
conv_name = 'unet'
args['hidden_classes'] = '0'

dataset_name = 'Vaihingen'
epoch = 1200
min_pixels = 50
reg_covar = 0.0001
args['n_components'] = 4
dataset_path='../datasets/'

hidden = []
if '_' in args['hidden_classes']:
    hidden = [int(h) for h in args['hidden_classes'].split('_')]
else:
    hidden = [int(args['hidden_classes'])]

num_known_classes = num_classes=args['num_classes'] - len(hidden)
num_unknown_classes = len(hidden)

if dataset_name == 'Potsdam':
    args['epoch_num'] = 600
    args['test_freq'] = 600
    args['save_freq'] = 600
    args['num_workers'] = 0
    epoch = 600

In [5]:
exp_group = conv_name + '_' + dataset_name + '_opengmm_dsm_' + str(min_pixels) + '_' + str(reg_covar) + '_' + str(args['n_components'])
exp_name = conv_name + '_' + dataset_name + '_opengmm_dsm_' + args['hidden_classes']


pretrained_path = os.path.join(ckpt_path, exp_name.replace('opengmm', 'base'), 'model_' + str(epoch) + '.pth')

# Setting device [0|1|2].
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
args['device'] = device


In [6]:
if (conv_name == 'unet'):

    net = UNet(4, num_classes=args['num_classes'], hidden_classes=hidden).to(args['device'])

elif (conv_name == 'fcnresnext50'):

    net = FCNResNext50(4, num_classes=args['num_classes'], pretrained=False, skip=True, hidden_classes=hidden).to(args['device'])

elif (conv_name == 'fcnwideresnet50'):

    net = FCNWideResNet50(4, num_classes=args['num_classes'], pretrained=False, skip=True, hidden_classes=hidden).to(args['device'])

elif (conv_name == 'fcndensenet121'):

    net = FCNDenseNet121(4, num_classes=args['num_classes'], pretrained=False, skip=True, hidden_classes=hidden).to(args['device'])
        

In [7]:
def test(test_loader, net, criterion, epoch, num_known_classes, num_unknown_classes, hidden, train_args, save_images, save_model, model_list):

    # Setting network for evaluation mode.
    net.eval()
    
    with torch.no_grad():

        # Creating output directory.
        check_mkdir(os.path.join(outp_path, exp_group, exp_name, 'epoch_' + str(epoch)))

        # Iterating over batches.
        for i, data in enumerate(test_loader):
            
            print('Test Batch %d/%d' % (i + 1, len(test_loader)))
            sys.stdout.flush()

            # Obtaining images, labels and paths for batch.
            inps_batch, labs_batch, true_batch, img_name = data

            inps_batch = inps_batch.squeeze()
            labs_batch = labs_batch.squeeze()
            true_batch = true_batch.squeeze()

            # Iterating over patches inside batch.
            for j in range(inps_batch.size(0)):
            
                print('    Test MiniBatch %d/%d' % (j + 1, inps_batch.size(0)))
                sys.stdout.flush()
                
                tic = time.time()
                
                for k in range(inps_batch.size(1)):
                    
                    inps = inps_batch[j, k].unsqueeze(0)
                    labs = labs_batch[j, k].unsqueeze(0)
                    true = true_batch[j, k].unsqueeze(0)

                    # Casting tensors to cuda.
                    inps, labs, true = inps.to(args['device']), labs.to(args['device']), true.to(args['device'])

                    # Casting to cuda variables.
                    inps = Variable(inps).to(args['device'])
                    labs = Variable(labs).to(args['device'])
                    true = Variable(true).to(args['device'])

                    # Forwarding.
                    if conv_name == 'unet':
                        outs, dec1, dec2, dec3, dec4 = net(inps, feat=True)
                    elif conv_name == 'fcndensenet121':
                        outs, classif1, fv2 = net(inps, feat=True)
                    elif conv_name == 'fcnwideresnet50':
                        outs, classif1, fv2 = net(inps, feat=True)

                    # Computing probabilities.
                    soft_outs = F.softmax(outs, dim=1)

                    # Obtaining prior predictions.
                    prds = soft_outs.data.max(1)[1]

                    # Obtaining posterior predictions.
                    if conv_name == 'unet':
                        feat_flat = torch.cat([outs, dec1, dec2, dec3], 1)
                    elif conv_name == 'fcndensenet121':
                        feat_flat = torch.cat([outs, classif1, fv2], 1)
                    elif conv_name == 'fcnwideresnet50':
                        feat_flat = torch.cat([outs, classif1, fv2], 1)
                        
                    feat_flat = feat_flat.permute(0, 2, 3, 1).contiguous().view(feat_flat.size(0) * feat_flat.size(2) * feat_flat.size(3), feat_flat.size(1)).cpu().numpy()
                    prds_flat = prds.cpu().numpy().ravel()
                    true_flat = true.cpu().numpy().ravel()

                    prds_post, scores = pred_pixelwise(model_list, feat_flat, prds_flat, num_known_classes, args['open_threshold'])
                    prds_post = prds_post.reshape(prds.size(0), prds.size(1), prds.size(2))
                    scores = scores.reshape(prds.size(0), prds.size(1), prds.size(2))

                    # Appending images for epoch loss calculation.
                    inps_np = inps.detach().squeeze(0).cpu().numpy()
                    labs_np = labs.detach().squeeze(0).cpu().numpy()
                    true_np = true.detach().squeeze(0).cpu().numpy()

                    # Saving predictions.
                    if (save_images):

                        pred_prev_path = os.path.join(outp_path, exp_group, exp_name, 'epoch_' + str(epoch), img_name[0].replace('.tif', '_prev_' + str(j) + '_' + str(k) + '.png'))
                        scor_path = os.path.join(outp_path, exp_group, exp_name, 'epoch_' + str(epoch), img_name[0].replace('.tif', '_scor_' + str(j) + '_' + str(k) + '.npy'))
                        
                        io.imsave(pred_prev_path, util.img_as_ubyte(prds.cpu().squeeze().numpy()))
                        np.save(scor_path, scores.squeeze())
                
                toc = time.time()
                print('        Elapsed Time: %.2f' % (toc - tic)) 
                
        sys.stdout.flush()

In [8]:
def validate(val_loader, net, criterion, epoch, num_known_classes, num_unknown_classes, hidden, train_args):
    
    model_list = []
    feat_list = []
    prds_list = []
    true_list = []
    
    # Setting network for evaluation mode.
    net.eval()
    
    count = 0
    
    n_patches = 0
    if dataset_name == 'Vaihingen':
        n_patches = 907 # Vaihingen.
    elif dataset_name == 'Potsdam':
        n_patches = 12393 # 8993 # Potsdam.
        
    np.random.seed(12345)
    #perm = np.random.permutation(n_patches)[:250].tolist()
    #perm = np.random.permutation(n_patches).tolist()
    label_count = {}
#     perm = np.random.permutation(n_patches)[:n_patches // 4].tolist()
    max_class = 0
    new_perm = []
    with torch.no_grad():

        for i, data in enumerate(val_loader):
            inps_batch, labs_batch, true_batch, img_name = data

            labs_batch = labs_batch.squeeze()

            for j in range(labs_batch.size(0)):
                for k in range(labs_batch.size(1)):
                    labs = labs_batch[j, k]
                    unique_counter = np.unique(labs,return_counts=True)
                    if len(unique_counter[0])>2:
                        aaaa=1
                    max_class = np.max(unique_counter[0]) if max_class<np.max(unique_counter[0]) else max_class

                    label_count[count] = unique_counter 
                    count += 1

    perm = np.random.permutation(count).tolist()

    n_classes_final = args['num_classes'] - len(args['hidden_classes'].split('_'))
    summation = np.zeros(max_class+1)
    for i in range(0,len(perm)):
        b = perm[i]
        new_perm.append(b)
        selected_patch=label_count[b]
        for j in range(0,len(selected_patch[0])):
            summation[selected_patch[0][j]] += selected_patch[1][j]
        if np.min(summation[0:n_classes_final])>=min_pixels and len(new_perm)>=200:
            break
    print("Pixel count per class:",summation)
    print(str(len(new_perm))," patches added.")
    count = 0
    perm=new_perm

    with torch.no_grad():

        # Creating output directory.
        check_mkdir(os.path.join(outp_path, exp_group, exp_name, 'epoch_' + str(epoch)))
        for i, data in enumerate(val_loader):
            
            print('Validation Batch %d/%d' % (i + 1, len(val_loader)))
            sys.stdout.flush()
            
            # Obtaining images, labels and paths for batch.
            inps_batch, labs_batch, true_batch, img_name = data
            
            inps_batch = inps_batch.squeeze()
            labs_batch = labs_batch.squeeze()
            true_batch = true_batch.squeeze()
            
            # Iterating over patches inside batch.
            for j in range(inps_batch.size(0)):
                
                for k in range(inps_batch.size(1)):
                    
                    if count in perm:
                        
                        inps = inps_batch[j, k].unsqueeze(0)
                        labs = labs_batch[j, k].unsqueeze(0)
                        true = true_batch[j, k].unsqueeze(0)

                        # Casting tensors to cuda.
                        inps, labs, true = inps.to(args['device']), labs.to(args['device']), true.to(args['device'])

                        # Casting to cuda variables.
                        inps = Variable(inps).to(args['device'])
                        labs = Variable(labs).to(args['device'])
                        true = Variable(true).to(args['device'])

                        # Forwarding.
                        if conv_name == 'unet':
                            outs, dec1, dec2, dec3, dec4 = net(inps, feat=True)
                        elif conv_name == 'fcndensenet121':
                            outs, classif1, fv2 = net(inps, feat=True)
                        elif conv_name == 'fcnwideresnet50':
                            outs, classif1, fv2 = net(inps, feat=True)

                        # Computing loss.
                        soft_outs = F.softmax(outs, dim=1)

                        # Obtaining predictions.
                        prds = soft_outs.data.max(1)[1]
                        
                        if conv_name == 'unet':
                            feat_flat = torch.cat([outs.squeeze(), dec1.squeeze(), dec2.squeeze(), dec3.squeeze()], 0)
                        elif conv_name == 'fcndensenet121':
                            feat_flat = torch.cat([outs.squeeze(), classif1.squeeze(), fv2.squeeze()], 0)
                        elif conv_name == 'fcnwideresnet50':
                            feat_flat = torch.cat([outs.squeeze(), classif1.squeeze(), fv2.squeeze()], 0)
                
                        feat_flat = feat_flat.permute(1, 2, 0).contiguous().view(feat_flat.size(1) * feat_flat.size(2), feat_flat.size(0)).cpu().numpy()
                        prds_flat = prds.cpu().numpy().ravel()
                        true_flat = true.cpu().numpy().ravel()
                    
                        feat_list.append(feat_flat)
                        prds_list.append(prds_flat)
                        true_list.append(true_flat)
                    
                    count = count + 1
        
        print('count', count)
        
        gc.collect()
        feat_list = np.asarray(feat_list)
        gc.collect()
        prds_list = np.asarray(prds_list)
        gc.collect()
        true_list = np.asarray(true_list)
        gc.collect()
        
        feat_list = feat_list.reshape(feat_list.shape[0] * feat_list.shape[1], feat_list.shape[2])
        gc.collect()
        prds_list = prds_list.reshape(prds_list.shape[0] * prds_list.shape[1])
        gc.collect()
        true_list = true_list.reshape(true_list.shape[0] * true_list.shape[1])
        gc.collect()
        
        for c in range(num_known_classes):
            
            print('Fitting model for class %d...' % (c))
            sys.stdout.flush()
            
            tic = time.time()
            
            # Computing GMM models from features.
            print(c,feat_list.shape,true_list.shape,prds_list.shape)
            model = fit_gmm_model(feat_list, true_list, prds_list, c, args['n_components'])
            model_list.append(model)
            
            toc = time.time()
            print('    Time spent fitting model %d: %.2f' % (c, toc - tic))
            
            # Saving model on disk.
            model_path = os.path.join(outp_path, exp_group, exp_name, 'model_' + str(c) + '.pkl')
            print('    Saving model at "%s"...' % (model_path))
            joblib.dump(model, model_path)
            
    return model_list

In [9]:
print('Loading pretrained weights from file "' + pretrained_path + '"')
if torch.cuda.is_available():
    if conv_name == 'fcnwideresnet50' and args['hidden_classes'] == '4':
        state_dict = torch.load(pretrained_path)

        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:] # remove module.
            new_state_dict[name] = v

        net.load_state_dict(new_state_dict)
    else:
        net.load_state_dict(torch.load(pretrained_path))
else:
    net.load_state_dict(torch.load(pretrained_path,map_location=torch.device('cpu') ))

args['best_record'] = {'epoch': 0, 'lr': 1e-4, 'val_loss': 1e10, 'acc': 0, 'acc_cls': 0, 'iou': 0}

# Setting datasets.
val_set = ListDataset(dataset_name, 'Validate', (args['h_size'], args['w_size']), 'statistical', hidden, overlap=False, use_dsm=True, dataset_path=dataset_path)
val_loader = DataLoader(val_set, batch_size=1, num_workers=args['num_workers'], shuffle=False)

test_set = ListDataset(dataset_name, 'Test', (args['h_size'], args['w_size']), 'statistical', hidden, overlap=True, use_dsm=True, dataset_path=dataset_path)
test_loader = DataLoader(test_set, batch_size=1, num_workers=args['num_workers'], shuffle=False)

# Setting criterion.
criterion = CrossEntropyLoss2d(size_average=False, ignore_index=5).to(args['device'])

# Making sure checkpoint and output directories are created.
check_mkdir(ckpt_path)
check_mkdir(os.path.join(ckpt_path, exp_group))
check_mkdir(os.path.join(ckpt_path, exp_group, exp_name))
check_mkdir(outp_path)
check_mkdir(os.path.join(outp_path, exp_group))
check_mkdir(os.path.join(outp_path, exp_group, exp_name))

from torch.autograd import Variable
from sklearn import metrics
from sklearn import mixture


# Validation function.
model_list = validate(val_loader, net, criterion, epoch, num_known_classes, num_unknown_classes, hidden, args)
#model_list = load_models(num_known_classes, outp_path, exp_group, exp_name)

# Computing test.
test(test_loader, net, criterion, epoch, num_known_classes, num_unknown_classes, hidden, args, True, epoch % args['save_freq'] == 0, model_list)

print("finished")

Loading pretrained weights from file "./ckpt/unet_Vaihingen_base_dsm_0/model_1200.pth"
self.n_classes 4
self.hidden_classes [0]
self.n_classes 4
self.hidden_classes [0]
Pixel count per class: [ 885284. 1345728. 1789171.   15447. 1383378.]
108  patches added.
Validation Batch 1/1
count 108
Fitting model for class 0...
0 (5419008, 260) (5419008,) (5419008,)
Initialization 0
Initialization converged: True	 time lapse 31.69472s	 ll 631.34019
    Time spent fitting model 0: 49.57
    Saving model at "./outputs/unet_Vaihingen_opengmm_dsm_50_0.0001_4/unet_Vaihingen_opengmm_dsm_0/model_0.pkl"...
Fitting model for class 1...
1 (5419008, 260) (5419008,) (5419008,)
Initialization 0
Initialization converged: True	 time lapse 62.04460s	 ll 602.29356
    Time spent fitting model 1: 87.54
    Saving model at "./outputs/unet_Vaihingen_opengmm_dsm_50_0.0001_4/unet_Vaihingen_opengmm_dsm_0/model_1.pkl"...
Fitting model for class 2...
2 (5419008, 260) (5419008,) (5419008,)
Initialization 0
Initialization 

/tmp/ipykernel_3263/2779408559.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  scores = np.zeros_like(prds_np, dtype=np.float)
/usr/local/lib/python3.8/dist-packages/skimage/util/dtype.py:541: UserWarning: Downcasting int64 to uint8 without scaling because max value 2 fits in uint8
  return _convert(image, np.uint8, force_copy)
/tmp/ipykernel_3263/1791890801.py:87: UserWarning: ./outputs/unet_Vaihingen_opengmm_dsm_50_0.0001_4/unet_Vaihingen_opengmm_dsm_0/epoch_1200/top_mosaic_09cm_area11_prev_0_0.png is a low contrast image
  io.imsave(pred_prev_path, util.img_as_ubyte(prds.cpu().squeeze().numpy()))
/tmp/ipykernel_3263/2779408559.py:12: DeprecationWarning: 

/usr/local/lib/python3.8/dist-packages/skimage/util/dtype.py:541: UserWarning: Downcasting int64 to uint8 without scaling because max value 2 fits in uint8
  return _convert(image, np.uint8, force_copy)
/tmp/ipykernel_3263/1791890801.py:87: UserWarning: ./outputs/unet_Vaihingen_opengmm_dsm_50_0.0001_4/unet_Vaihingen_opengmm_dsm_0/epoch_1200/top_mosaic_09cm_area11_prev_0_9.png is a low contrast image
  io.imsave(pred_prev_path, util.img_as_ubyte(prds.cpu().squeeze().numpy()))
/tmp/ipykernel_3263/2779408559.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  scores = np.zeros_like(prds_np, dtype=np.float)
/usr/local/lib/python3.8/dist-packages/skimage/util/dtype.

        Elapsed Time: 24.73
    Test MiniBatch 2/23


/usr/local/lib/python3.8/dist-packages/skimage/util/dtype.py:541: UserWarning: Downcasting int64 to uint8 without scaling because max value 2 fits in uint8
  return _convert(image, np.uint8, force_copy)
/tmp/ipykernel_3263/1791890801.py:87: UserWarning: ./outputs/unet_Vaihingen_opengmm_dsm_50_0.0001_4/unet_Vaihingen_opengmm_dsm_0/epoch_1200/top_mosaic_09cm_area11_prev_0_16.png is a low contrast image
  io.imsave(pred_prev_path, util.img_as_ubyte(prds.cpu().squeeze().numpy()))
/tmp/ipykernel_3263/2779408559.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  scores = np.zeros_like(prds_np, dtype=np.float)
/usr/local/lib/python3.8/dist-packages/skimage/util/dtype

...

        Elapsed Time: 14.84
    Test MiniBatch 23/23


/usr/local/lib/python3.8/dist-packages/skimage/util/dtype.py:541: UserWarning: Downcasting int64 to uint8 without scaling because max value 2 fits in uint8
  return _convert(image, np.uint8, force_copy)
/tmp/ipykernel_3263/1791890801.py:87: UserWarning: ./outputs/unet_Vaihingen_opengmm_dsm_50_0.0001_4/unet_Vaihingen_opengmm_dsm_0/epoch_1200/top_mosaic_09cm_area34_prev_21_12.png is a low contrast image
  io.imsave(pred_prev_path, util.img_as_ubyte(prds.cpu().squeeze().numpy()))
/tmp/ipykernel_3263/2779408559.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  scores = np.zeros_like(prds_np, dtype=np.float)
/usr/local/lib/python3.8/dist-packages/skimage/util/dtyp

/usr/local/lib/python3.8/dist-packages/skimage/util/dtype.py:541: UserWarning: Downcasting int64 to uint8 without scaling because max value 2 fits in uint8
  return _convert(image, np.uint8, force_copy)
/tmp/ipykernel_3263/1791890801.py:87: UserWarning: ./outputs/unet_Vaihingen_opengmm_dsm_50_0.0001_4/unet_Vaihingen_opengmm_dsm_0/epoch_1200/top_mosaic_09cm_area34_prev_22_8.png is a low contrast image
  io.imsave(pred_prev_path, util.img_as_ubyte(prds.cpu().squeeze().numpy()))
/tmp/ipykernel_3263/2779408559.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  scores = np.zeros_like(prds_np, dtype=np.float)
/usr/local/lib/python3.8/dist-packages/skimage/util/dtype

        Elapsed Time: 14.04


/usr/local/lib/python3.8/dist-packages/skimage/util/dtype.py:541: UserWarning: Downcasting int64 to uint8 without scaling because max value 2 fits in uint8
  return _convert(image, np.uint8, force_copy)
/tmp/ipykernel_3263/1791890801.py:87: UserWarning: ./outputs/unet_Vaihingen_opengmm_dsm_50_0.0001_4/unet_Vaihingen_opengmm_dsm_0/epoch_1200/top_mosaic_09cm_area34_prev_22_12.png is a low contrast image
  io.imsave(pred_prev_path, util.img_as_ubyte(prds.cpu().squeeze().numpy()))
